In [1]:
!pip install selenium
!pip install webdriver-manager

In [ ]:
import json
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import pandas as pd
def get_cik_data():
    url = "https://www.sec.gov/files/company_tickers_exchange.json"
    headers = {'User-Agent': 'AgentIdea (Contact: chanathip.thc@gmail.com)'}
    try:
        resp = requests.get(url, headers=headers)
        resp.raise_for_status()
        data = resp.json()
        fields = data.get('fields', [])
        cik_index = -1
        ticker_index = -1
        if 'cik' in fields:
            cik_index = fields.index('cik')
        elif 'cik_str' in fields:
            cik_index = fields.index('cik_str')
        if 'ticker' in fields:
            ticker_index = fields.index('ticker')
        company_data = data.get('data', [])
        cik_list = []
        ticker_list = []

        for item in company_data:
            cik = None
            ticker = None
            if cik_index != -1 and len(item) > cik_index:
                cik = str(item[cik_index]).zfill(10)
                cik_list.append(cik)
            if ticker_index != -1 and len(item) > ticker_index:
                ticker = item[ticker_index]
                ticker_list.append(ticker)
            else:
                ticker_list.append(None)

        df = pd.DataFrame({
            'cik_str': cik_list,
            'ticker': ticker_list,
        })
        return df

    except requests.exceptions.RequestException as e:
        print(f"เกิดข้อผิดพลาดในการดึงข้อมูล JSON: {e}")
        return None
    except Exception as e:
        print(f"เกิดข้อผิดพลาดโดยรวม: {e}")
        return None

def download_financial_statements(cik, ticker):
    base_url = "https://site.financialmodelingprep.com/financial-statements/"
    company_url = f"{base_url}{ticker}"
    output_folder = "downloaded_csvs"
    os.makedirs(output_folder, exist_ok=True)

    chrome_options = Options()
    prefs = {"download.default_directory": os.path.abspath(output_folder)}
    chrome_options.add_experimental_option("prefs", prefs)
    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        driver.get(company_url)
        wait = WebDriverWait(driver, 1)
        time.sleep(2)  # รอให้หน้าเว็บโหลด

        statements_to_download = ["Income Statement", "Cash Flow Statement", "Balance Sheet"]
        tab_index = 1

        for statement_name in statements_to_download:
            try:
                # คลิกที่แท็บของงบการเงินนั้นๆ ก่อน
                tab_xpath = f"/html/body/div[1]/div/div[2]/div[2]/div[2]/div/div/div[1]/div/div[2]/button[{tab_index}]"
                tab_button = wait.until(EC.element_to_be_clickable((By.XPATH, tab_xpath)))
                tab_button.click()
                time.sleep(2)  # รอให้เนื้อหาของแท็บโหลด

                # จากนั้นคลิกปุ่มดาวน์โหลด CSV ในแท็บนั้น
                download_button_xpath = "/html/body/div[1]/div/div[2]/div[2]/div[2]/div/div/div[1]/div/button[2]"
                download_button = wait.until(EC.element_to_be_clickable((By.XPATH, download_button_xpath)))
                download_button.click()
                print(f"ดาวน์โหลด {statement_name} CSV สำหรับ Ticker: {ticker} CIK: {cik} แล้ว")
                time.sleep(2)  # รอให้การดาวน์โหลดเสร็จสิ้น
            except Exception as e:
                print(f"ไม่พบหรือเกิดข้อผิดพลาดในการดาวน์โหลด {statement_name} CSV สำหรับ Ticker: {ticker} CIK: {cik} - {e}")

            tab_index += 1

    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการดำเนินการสำหรับ Ticker: {ticker} CIK: {cik} - {e}")
    finally:
        driver.quit()

if __name__ == "__main__":
    cik_df = get_cik_data()
    if cik_df is not None:
        print("ดึงข้อมูล CIK และ Ticker สำเร็จ:")
        print(cik_df.head())

        print(f"\nกำลังดาวน์โหลด financial statements สำหรับ {len(cik_df)} บริษัท...")

        for index, row in cik_df.iterrows():
            cik = row['cik_str']
            ticker = row['ticker']
            download_financial_statements(cik, ticker)

        print("\nการดำเนินการดาวน์โหลด financial statements เสร็จสิ้น")

ดึงข้อมูล CIK และ Ticker สำเร็จ:
      cik_str ticker
0  0000320193   AAPL
1  0000789019   MSFT
2  0001045810   NVDA
3  0001652044  GOOGL
4  0001018724   AMZN

กำลังดาวน์โหลด financial statements สำหรับ 9691 บริษัท...

(1/9691) กำลังดาวน์โหลดสำหรับ Ticker: AAPL
ดาวน์โหลด Income Statement CSV สำหรับ Ticker: AAPL CIK: 0000320193 แล้ว
ดาวน์โหลด Cash Flow Statement CSV สำหรับ Ticker: AAPL CIK: 0000320193 แล้ว
ดาวน์โหลด Balance Sheet CSV สำหรับ Ticker: AAPL CIK: 0000320193 แล้ว

(2/9691) กำลังดาวน์โหลดสำหรับ Ticker: MSFT
ดาวน์โหลด Income Statement CSV สำหรับ Ticker: MSFT CIK: 0000789019 แล้ว
ดาวน์โหลด Cash Flow Statement CSV สำหรับ Ticker: MSFT CIK: 0000789019 แล้ว
ดาวน์โหลด Balance Sheet CSV สำหรับ Ticker: MSFT CIK: 0000789019 แล้ว
